In [1]:
import psycopg2
import pandas as pd
import pandas.io.sql as sqlio

In [2]:
conn = psycopg2.connect(
    host="localhost",
    database="postgres",
    user="postgres",
    password="password")

In [3]:
cur = conn.cursor()

In [4]:
print('PostgreSQL database version:')
cur.execute('SELECT version()')
db_version = cur.fetchone()
print(db_version)

PostgreSQL database version:
('PostgreSQL 15.1, compiled by Visual C++ build 1914, 64-bit',)


In [ ]:
## cell for getting student/team names
data = sqlio.read_sql_query("""select distinct * from users
order by userid
 """, conn)

data.head(30)

In [ ]:
# queries
#cur.execute("select * from users;")
#cur.fetchmany(10)

# directly into pandas dataframe
#data = sqlio.read_sql_query("SELECT * FROM users", conn)
#data.head(15)

## count amount of test runs per users
#data = sqlio.read_sql_query("select username, count(userid) c from testrun natural join users group by username order by c desc;", conn)
#data.head(10)

## count successful test run for a specific user


data = sqlio.read_sql_query("""SELECT count(testname) test_run_count, username, suitename, testname
    FROM testrun
        NATURAL JOIN users
        natural join testintestrun
        natural join test
        natural join suitelookup
        natural join testnamelookup
    where username like '%-anonymized'
    group by username, suitename, testname
    order by suitename
    limit 1000""" , conn)

data

In [ ]:
#random small queries
data = sqlio.read_sql_query("""select distinct testnameid, suiteid, suitename, testname from test
natural join testnamelookup
natural join suitelookup
order by testnameid
 """, conn)
#data.to_csv('testnames_and_suitenames.csv', index=False,)
data


In [ ]:
#random small queries 2
data = sqlio.read_sql_query("""select username, userid, count(userid) testruns from testrun 
    natural join users 
    group by username, userid
    order by testruns desc;
 """, conn)
#data.to_csv('total_test_runs_by_group.csv', index=False,)
#data#.pivot(index='username', columns='testruns')

tt = sqlio.read_sql_query("""SELECT count(username) test_run_count, username, suitename, suiteid
        FROM testrun
            NATURAL JOIN users
            natural join testintestrun
            natural join test
            natural join suitelookup
        where username like '%-anonymized'
        group by username, suitename, suiteid
        order by suiteid
        """, conn)

tt

#### code to add testsuites as columns to student index

In [ ]:
students = pd.read_csv('students_cleaned.csv',)
testsuites = pd.read_csv('testsuites.csv', )
newdf = students.reindex(columns=testsuites.suiteid.astype('int32'))
students = students.join(newdf)

#newdf.rename(columns={'suiteid'})
students.to_csv('students_and_testsuiteids.csv')
#for i in range(1-46):
#    students = students.astype('int32')
students



In [102]:
## add number of testsuites runs to students/groups

import sys

## main insertion method, loop this


students_iterate = students
data = sqlio.read_sql_query("""SELECT count(username) test_run_count, username, suitename, suiteid
        FROM testrun
            NATURAL JOIN users
            natural join testintestrun
            natural join test
            natural join suitelookup
        where username = %s
        group by username, suitename, suiteid
        order by suiteid
        """, conn, params = ['anonymized-anonymized'])


for name in students['username']:
    suiteids = data['suiteid'].tolist() # 1-45 sometimes 1-37 set
    data = sqlio.read_sql_query("""SELECT count(username) test_run_count, username, suitename, suiteid
        FROM testrun
            NATURAL JOIN users
            natural join testintestrun
            natural join test
            natural join suitelookup
        where username = %s
        group by username, suitename, suiteid
        order by suiteid
        """, conn, params = [name])

    for i in suiteids:
        cell = (data.loc[data['suiteid'] == i]['test_run_count'])
        if len(cell) == 1:
            count = cell.item()
        else:
            count = 0
        students.at[students['username'] == name, [i]] = count

#data

In [8]:
students = pd.read_csv('students_and_test_run_counts.csv', index_col=0)
#students
# only single students regex: '^[^-]+$'
#students[students['username'].str.contains('^[^-]+$')]
#students[students['username'].str.contains('-')]
#print(students.isna().sum()
#students.sort_values(by='username').loc[:,['username']].to_csv('usernames-sorted.csv')


In [ ]:
pd.read_csv('only_students_test_run_counts.csv', index_col=0,).fillna(0)

##### add run test counts from groups into individual student rows

In [ ]:
single_students = pd.read_csv('only_students_test_run_counts.csv', index_col=0, usecols=[i for i in range(1,48)]).fillna(0)
groups = pd.read_csv('only_groups_test_run_counts.csv', index_col=0, usecols=[i for i in range(1,48)]).fillna(0)
                                                                                                              
#single_students = pd.read_csv('only_students_test_run_counts.csv').fillna(0)
#groups = pd.read_csv('only_groups_test_run_counts.csv').fillna(0)

# do loop like this 
# first iterate names of single_students
# find corresponding group
# add together the test run counts for each suite
# append row to new dataframe copy of single_students
newdf = single_students.copy()
doubleteam = 0
for index, studentrow in single_students.iterrows():
    grouprow = groups[groups['username'].str.contains(studentrow['username'])]
    
    # check if grouprow is empty first
    if grouprow.shape[0] != 0:
        for column, content in grouprow.items():
            if column == 'username': # skip username row
                continue
            if content.shape != (1,):
                newdf.at[index, column] += (content.iat[0] + content.iat[1])
                doubleteam+=1
            else:
                newdf.at[index, column] += content
            
    # do pandas.apply here?
print(f'rows with double team: {doubleteam}\nteams with double students: {doubleteam/45}')
#print(groups[groups['username'].str.contains(studentrow['username'])])
    

testdf = pd.DataFrame(newdf.loc[[18640]]).append(groups.loc[[31492]]).append(single_students.loc[[18640]])
testdf


In [141]:
data = sqlio.read_sql_query("""SELECT count(username) test_run_count, username, suitename, suiteid
        FROM testrun
            NATURAL JOIN users
            natural join testintestrun
            natural join test
            natural join suitelookup
        where username = %s
        group by username, suitename, suiteid
        order by suiteid
        """, conn, params = ['anonymized'])


#data.loc[data['suiteid'] == 1]['test_run_count']
#data = data.pivot(index = None, columns = 'suiteid', values='test_run_count')
#data = data.drop(['suitename'], 1)
data.loc[data['suiteid'] == 5]['test_run_count']

#merged = pd.merge(students, data.T, how='left', left_index=True, right_index=True, )
#merged.get(92)



4    234
Name: test_run_count, dtype: int64

In [ ]:
## get grades from canvas and clean up

grades = pd.read_csv('grades.csv')
#grades.loc[:,['Student','Project submission (220629)']
grades = grades.rename(columns={'Project submission (anonymized)' : 'grade', 'SIS Login ID' : 'username', 'Student': 'name'})
#gradesdict = {5.0: 'A', 4.0 : 'B', 3.0 : 'C', 2.0 : 'D', 1.0 : 'E', 0.0 : 'F'}
gradespassfail = {5.0: 1, 4.0 : 1, 3.0 : 1, 2.0 : 1, 1.0 : 1, 0.0 : 0, 3.5: 1}
grades = grades.replace({'grade': gradespassfail})
#grades.loc[grades['grade'] >= 1, 'grade'] = 1

grades = grades.drop([0,1,252])
grades = grades.replace(to_replace='@kth.se', value = '', regex=True)
#grades.loc[grades['username'] == 'anonymized'].drop(columns = list(grades.filter(regex='(TW)|(Bonus)|(reflection)|([Uu]nposted)')))
#grades.filter(regex = '(name)|([gG]rade)|([Pp]roject)')
grades[grades['grade'] < 1 ]['grade'] #find null grades
grades[['username', 'grade']]


##### add grades to students

In [10]:
merge = pd.merge(grades.loc[:,['username','grade']], newdf, left_on = 'username', right_on = 'username', how = 'right')

#students.sort_values(by='username').loc[:,['username']].to_csv('usernames-sorted.csv')
#merge.loc[:,['username', 'grade']].to_csv('usernames_and_grades_sorted.csv')

merge[merge.isna().any(axis=1)] # null grades

merge = merge.fillna(0).astype({'grade': 'int32'})
merge = merge.drop(merge.loc[merge['username'] == 'anonymous'].index) # drop anon
merge.to_csv('final_training_data_test.csv')

In [11]:
# close connection
cur.close()
conn.close()